In [111]:
# Importar bibliotecas

import pandas as pd
from pulp import *

In [110]:
# Crear/leer datos

def leer_datos():
    # Aquí lógica para leer los datos desde un archivo o base de datos
    proyectos = pd.DataFrame({       
        'eslora': [120, 100],
        'manga': [18, 15]}, 
        index=['PRO1', 'PRO2'])

    periodos = pd.DataFrame({
        'tipo_desc': ['FLOTE', 'FLOTE'],
        'fecha_inicio': ['2025-08-08', '2025-08-10'],
        'fecha_fin': ['2025-08-20', '2025-08-16'],
        'proyecto_id': ['PRO1', 'PRO2'],
        'periodo_id': [0, 0]})

    periodos.index = periodos['proyecto_id'] + '_' + periodos['periodo_id'].astype(str)

    muelles = pd.DataFrame({
        'longitud': [130, 110],
        'ancho': [20, 20]},
        index=['SUR', 'NORTE'])

    fecha_inicial = periodos['fecha_inicio'].min()

    return proyectos, periodos, muelles, fecha_inicial


In [ ]:
# Preprocesar datos

def preprocesar_datos(proyectos, periodos, muelles, fecha_inicial):

    # Convertir fechas a integer
    periodos['fecha_inicio'] = pd.to_datetime(periodos['fecha_inicio'])
    periodos['fecha_fin'] = pd.to_datetime(periodos['fecha_fin'])
    fecha_inicial = periodos['fecha_inicio'].min()

    periodos['fecha_inicio'] = (periodos['fecha_inicio'] - fecha_inicial).dt.days
    periodos['fecha_fin'] = (periodos['fecha_fin'] - fecha_inicial).dt.days

    # Crear una lista de días
    dias = list(range(periodos['fecha_inicio'].min(), periodos['fecha_fin'].max()+1))

    return proyectos, periodos, muelles, dias

In [173]:
# Definir variables de decisión

def definir_variables(proyectos, periodos, muelles):
    dias_vars = {}
    locs_vars = {}
    
    # Definir variables para cada periodo solo en los días y localizaciones correspondientes

    for p in periodos.index:
        dias_vars[p] = list(range(periodos.loc[p, 'fecha_inicio'], periodos.loc[p, 'fecha_fin'] +1))
        locs_vars[p] = []
        if periodos.loc[p, 'tipo_desc'] == 'FLOTE':
            locs_vars[p].extend([m for m in muelles.index if (muelles.loc[m, 'longitud'] >= proyectos.loc[periodos.loc[p, 'proyecto_id'], 'eslora'] and 
                              muelles.loc[m, 'ancho'] >= proyectos.loc[periodos.loc[p, 'proyecto_id'], 'manga'])])

    x = {(p, d, loc): LpVariable("x_"+ p +"_" + str(d) + "_" + loc,(p, d, loc), cat='Binary')
         for p in periodos.index
         for d in dias_vars[p]
         for loc in locs_vars[p] if loc}
    
    return x, dias_vars, locs_vars

In [174]:
# Definir la función objetivo

def definir_funcion_objetivo(x):
    objetivo = lpSum(x.values())
    return objetivo

In [178]:
# Definir restricciones

def definir_restricciones(x, dias, dias_vars, locs_vars, periodos, muelles, proyectos):

    restricciones = {}

    # Cada proyecto como mucho en un muelle por día
    restricciones['1_por_día'] = []
    for p in periodos.index:
        for d in dias_vars[p]:
            restricciones['1_por_día'].append(lpSum(x[(p, d, m)] for m in locs_vars[p]) <= 1)
    
    # Los barcos en el mismo muelle no pueden exceder la longitud del muelle
    restricciones['Longitud_Muelle'] = []
    for d in dias:
        for m in muelles.index:
            restricciones['Longitud_Muelle'].append(lpSum(x[(d,p,m)] * proyectos.loc[periodos.loc[p,'proyecto_id'], 'eslora'] for p in periodos.index if (d,p,m) in x.keys()) <= muelles.loc[m, 'longitud'])

    return restricciones

In [167]:
# Resolver el problema de optimización

def resolver_problema(objetivo, restricciones):

    prob = LpProblem("Asignación de Periodos a Muelles", LpMaximize)
    prob += objetivo

    for constraint in restricciones.values():
        for c in constraint:
            prob += c

    prob.solve()

    return prob

In [168]:
# Imprimir asignación de proyectos por muelle

def imprimir_asignacion(prob, x_dpm, dias, periodos, muelles):
    
    print("Estado de la solución:", LpStatus[prob.status])
    print("\nAsignación de Proyectos a Muelles:\n")
    print("Día\t", "\t".join(muelles.index))

    for d in dias:
        row = f"{d}\t"
        for p in periodos.index:
            for m in muelles.index:
                if x_dpm[d][p][m].varValue == 1:
                    row += f"{p}\t"
                    break
        print(row)


In [179]:
# Optimize

def optimize():
    proyectos, periodos, muelles, fecha_inicial = leer_datos()
    proyectos, periodos, muelles, dias = preprocesar_datos(proyectos, periodos, muelles, fecha_inicial)
    
    x, dias_vars, locs_vars = definir_variables(proyectos, periodos, muelles)
    objetivo = definir_funcion_objetivo(x)
    restricciones = definir_restricciones(x, dias, dias_vars, locs_vars, periodos, muelles, proyectos)
    
    prob = resolver_problema(objetivo, restricciones)
    imprimir_asignacion(prob, x, dias, periodos, muelles)

In [180]:
optimize()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/miniconda3/envs/NAME_ENV/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/x4/4ffrr56n3jzb55xn5fzcplwc0000gn/T/db29939fcc994d729c948bf4fb074932-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/x4/4ffrr56n3jzb55xn5fzcplwc0000gn/T/db29939fcc994d729c948bf4fb074932-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 51 COLUMNS
At line 160 RHS
At line 207 BOUNDS
At line 235 ENDATA
Problem MODEL has 46 rows, 27 columns and 27 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 20 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -20 to -1.79769e+308
Probing was tried 0 times and created 0 cu

/opt/miniconda3/envs/NAME_ENV/lib/python3.10/site-packages/pulp/pulp.py:1455: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


KeyError: 0

In [155]:
proyectos, periodos, muelles, fecha_inicial = leer_datos()
proyectos, periodos, muelles, dias = preprocesar_datos(proyectos, periodos, muelles, fecha_inicial)
x = definir_variables(proyectos, periodos, muelles)
objetivo = definir_funcion_objetivo(x)
prob = LpProblem("Asignación de Periodos a Muelles", LpMaximize)
prob += objetivo

/opt/miniconda3/envs/NAME_ENV/lib/python3.10/site-packages/pulp/pulp.py:1455: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
